<a href="https://colab.research.google.com/github/leoroman1/DL_project/blob/main/DL_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# connect to your drive with the dataset folder

from google.colab import drive
drive.mount('/content/drive')

dataset_dir = '/content/drive/MyDrive/Task01_BrainTumour/Task01_BrainTumour'
images_dir = '/content/drive/MyDrive/Task01_BrainTumour/Task01_BrainTumour/imagesTr'
labels_dir = '/content/drive/MyDrive/Task01_BrainTumour/Task01_BrainTumour/labelsTr'

In [ ]:
# dataset tar extraction ONLY IF NEEDED

import tarfile
from tqdm import tqdm
import os

# Paths (modify as needed)
tar_path = '/content/drive/MyDrive/Task01_BrainTumour.tar'
destination_path = '/content/drive/MyDrive/Task01_BrainTumour'

# Ensure output directory exists
os.makedirs(destination_path, exist_ok=True)

# Open and extract with progress bar
with tarfile.open(tar_path) as tar:
    members = tar.getmembers()
    print(f"Extracting {len(members)} files to: {destination_path}")
    for member in tqdm(members, desc="Extracting", unit="file"):
        tar.extract(member, path=destination_path)

print("Extraction completed.")


TO DO:

. normalization fatta bene

. crop and back to original padding (capire se ha senso)

. fare dice loss bene e vedere come combinarla a cross entropy bene

. sistemare tutti i codici ordinati e con variabili ben nominate

In [ ]:
import os
import nibabel as nib

image_dir = "/content/drive/MyDrive/Task01_BrainTumour/Task01_BrainTumour/imagesTr"
label_dir = "/content/drive/MyDrive/Task01_BrainTumour/Task01_BrainTumour/labelsTr"

# Filter out hidden/system files like ._BRATS_001.nii.gz
def is_valid_nifti(fname):
    return fname.endswith(".nii.gz") and not fname.startswith("._")

# Get sorted lists of image and label paths
image_paths = sorted([
    os.path.join(image_dir, fname)
    for fname in os.listdir(image_dir)
    if is_valid_nifti(fname)
])

label_paths = sorted([
    os.path.join(label_dir, fname)
    for fname in os.listdir(label_dir)
    if is_valid_nifti(fname)
])

# Sanity check: print number of samples
print("Number of samples:", len(image_paths))

# Show dimensions and sample names for the first 10
for img_path, lbl_path in zip(image_paths[:10], label_paths[:10]):
    img = nib.load(img_path)
    lbl = nib.load(lbl_path)

    img_data = img.get_fdata()
    lbl_data = lbl.get_fdata()

    print(f"\nImage: {os.path.basename(img_path)} - Shape: {img_data.shape}")
    print(f"Label: {os.path.basename(lbl_path)} - Shape: {lbl_data.shape}")



In [ ]:
import os
import nibabel as nib
import matplotlib.pyplot as plt

image_dir = "/content/drive/MyDrive/Task01_BrainTumour/Task01_BrainTumour/imagesTr"
label_dir = "/content/drive/MyDrive/Task01_BrainTumour/Task01_BrainTumour/labelsTr"

def is_valid_nifti(fname):
    return fname.endswith(".nii.gz") and not fname.startswith("._")

image_paths = sorted([
    os.path.join(image_dir, fname)
    for fname in os.listdir(image_dir)
    if is_valid_nifti(fname)
])

label_paths = sorted([
    os.path.join(label_dir, fname)
    for fname in os.listdir(label_dir)
    if is_valid_nifti(fname)
])

# Load first image and label
img = nib.load(image_paths[0]).get_fdata()
lbl = nib.load(label_paths[0]).get_fdata()

num_slices = img.shape[2]
first_10 = list(range(50, 60))
last_10 = list(range(70, 80))
slices_to_show = first_10 + last_10

modalities = ['T1', 'T1c', 'T2', 'FLAIR']

for slice_idx in slices_to_show:
    fig, axs = plt.subplots(1, 5, figsize=(20, 4))
    fig.suptitle(f"Slice {slice_idx}")

    for ch in range(4):
        slice_img = img[:, :, slice_idx, ch]
        slice_img_norm = (slice_img - slice_img.min()) / (slice_img.max() - slice_img.min() + 1e-8)
        axs[ch].imshow(slice_img_norm, cmap='gray')
        axs[ch].set_title(modalities[ch])
        axs[ch].axis('off')

    # Show label mask as last image in the row
    axs[4].imshow(lbl[:, :, slice_idx], cmap='nipy_spectral', vmin=0, vmax=4)
    axs[4].set_title('Label')
    axs[4].axis('off')

    plt.show()


In [ ]:
import os
import nibabel as nib
import numpy as np

label_dir = "/content/drive/MyDrive/Task01_BrainTumour/Task01_BrainTumour/labelsTr"

def is_valid_nifti(fname):
    return fname.endswith(".nii.gz") and not fname.startswith("._")

label_paths = sorted([
    os.path.join(label_dir, fname)
    for fname in os.listdir(label_dir)
    if is_valid_nifti(fname)
])

# Load first label mask
label_img = nib.load(label_paths[0])
label_data = label_img.get_fdata()

# Print unique values and their counts
unique_vals, counts = np.unique(label_data, return_counts=True)

print("Unique label values and their counts:")
for val, count in zip(unique_vals, counts):
    print(f"Label {int(val)}: {count} voxels")


In [27]:
import os
import shutil

# Paths
simple_test_images_dir = '/content/drive/MyDrive/Task01_BrainTumour/Task01_BrainTumour/samples_imagesTr'
simple_test_labels_dir = '/content/drive/MyDrive/Task01_BrainTumour/Task01_BrainTumour/samples_labelsTr'

# Create dirs if not exist
os.makedirs(simple_test_images_dir, exist_ok=True)
os.makedirs(simple_test_labels_dir, exist_ok=True)

# Get first 25 files (excluding hidden/system files)
images_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.nii.gz') and not f.startswith('._')])[:25]



for f in images_files:
    src_img = os.path.join(images_dir, f)
    dst_img = os.path.join(simple_test_images_dir, f)
    if not os.path.exists(dst_img):
        shutil.copy(src_img, dst_img)
    else:
        print(f"Image file {f} already exists, skipping.")

    src_label = os.path.join(labels_dir, f)
    dst_label = os.path.join(simple_test_labels_dir, f)
    if not os.path.exists(dst_label):
        shutil.copy(src_label, dst_label)
    else:
        print(f"Label file {f} already exists, skipping.")


In [45]:
from tensorflow.keras import layers, models

input_layer = layers.Input(shape=(240, 240, 4))  # 4 channels
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_layer)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
output_layer = layers.Conv2D(4, (1, 1), activation='softmax')(x)  # softmax for multi-class

model = models.Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss=combined_loss, metrics=['accuracy'])



In [44]:
import os
import numpy as np
import nibabel as nib
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras import backend as K

class BrainTumor2DSliceGenerator(Sequence):
    def __init__(self, image_dir, label_dir, batch_size=8, slice_step=1, shuffle=True):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.slice_step = slice_step
        # Filter out files starting with '._'
        self.patient_ids = sorted([f.replace('.nii.gz', '') for f in os.listdir(image_dir) if not f.startswith('._')])
        self.indexes = self._create_index_list()
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def _create_index_list(self):
        index_list = []
        for pid in self.patient_ids:
            img_path = os.path.join(self.image_dir, pid + '.nii.gz')
            img = nib.load(img_path).get_fdata()  # shape: H x W x D x C
            for i in range(0, img.shape[2], self.slice_step):
                index_list.append((pid, i))
        return index_list

    def __len__(self):
        return int(np.ceil(len(self.indexes) / self.batch_size))

    def __getitem__(self, idx):
        batch_indexes = self.indexes[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_X = []
        batch_y = []
        for pid, slice_idx in batch_indexes:
            img = nib.load(os.path.join(self.image_dir, pid + '.nii.gz')).get_fdata()
            label = nib.load(os.path.join(self.label_dir, pid + '.nii.gz')).get_fdata()
            img_slice = img[:, :, slice_idx, :]  # shape: H x W x C
            label_slice = label[:, :, slice_idx]  # shape: H x W

            # Normalize and cast
            img_slice = (img_slice - np.mean(img_slice)) / (np.std(img_slice) + 1e-5)
            batch_X.append(img_slice)
            batch_y.append(label_slice)

        return np.array(batch_X), np.array(batch_y).astype(np.int32)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)


def dice_loss_per_class(y_true, y_pred, num_classes=4, smooth=1e-6):
    """
    Computes Dice loss per class and averages over all classes.
    Accepts sparse integer labels (y_true) and softmax outputs (y_pred).
    """
    if len(y_true.shape) == 3:
        y_true = tf.expand_dims(y_true, axis=-1)

    y_true_one_hot = tf.one_hot(tf.cast(y_true[..., 0], tf.int32), depth=num_classes)

    dice = 0
    for c in range(num_classes):
        y_true_c = y_true_one_hot[..., c]
        y_pred_c = y_pred[..., c]
        intersection = tf.reduce_sum(y_true_c * y_pred_c)
        denominator = tf.reduce_sum(y_true_c) + tf.reduce_sum(y_pred_c)
        dice += (2. * intersection + smooth) / (denominator + smooth)

    return 1 - dice / num_classes

def combined_loss(y_true, y_pred):
    ce_loss = tf.keras.losses.SparseCategoricalCrossentropy()(y_true, y_pred)
    dice = dice_loss_per_class(y_true, y_pred, num_classes=4)
    return ce_loss + dice




In [ ]:
simple_test_data_gen = BrainTumor2DSliceGenerator(
    simple_test_images_dir,
    simple_test_labels_dir,
    batch_size=8,
    slice_step=4,
    shuffle=True
)

model.fit(simple_test_data_gen, epochs=3)


In [ ]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Define number of classes
num_classes = 4

# Define a discrete colormap with distinct colors (can customize colors)
colors = ['black', 'green', 'yellow', 'red']  # one color per class, adjust as you like
cmap = mcolors.ListedColormap(colors)

# Define boundaries between classes for normalization
bounds = list(range(num_classes + 1))  # [0,1,2,3,4]
norm = mcolors.BoundaryNorm(bounds, cmap.N)

# Set paths

test_volume_path = '/content/drive/MyDrive/Task01_BrainTumour/Task01_BrainTumour/samples_imagesTr/BRATS_001.nii.gz'
label_path       = '/content/drive/MyDrive/Task01_BrainTumour/Task01_BrainTumour/samples_labelsTr/BRATS_001.nii.gz'

# Load volume & label

img = nib.load(test_volume_path).get_fdata()   # shape: H x W x D x 4
label = nib.load(label_path).get_fdata().astype(np.int32)  # shape: H x W x D


# Customize range of slices

start_slice = 45
end_slice = 50


# Predict and show

for i in range(start_slice, end_slice):
    # Extract the slice (H x W x 4)
    img_slice = img[:, :, i, :]
    label_slice = label[:, :, i]

    # Preprocess (same as training)
    img_slice_norm = (img_slice - np.mean(img_slice)) / (np.std(img_slice) + 1e-5)

    # Add batch dimension
    input_slice = np.expand_dims(img_slice_norm, axis=0)  # shape: 1 x H x W x 4

    # Predict
    pred = model.predict(input_slice, verbose=0)
    pred_class = np.argmax(pred[0], axis=-1)  # shape: H x W

    # Plot
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.imshow(img_slice[:, :, 0], cmap='gray')  # Show channel 0 (e.g., T1)
    plt.title(f'Slice {i} - MRI (T1)')

    plt.subplot(1, 3, 2)
    plt.imshow(label_slice, cmap=cmap, norm=norm)
    plt.title('Ground Truth')

    plt.subplot(1, 3, 3)
    plt.imshow(pred_class, cmap=cmap, norm=norm)
    plt.title('Model Prediction')


    plt.tight_layout()
    plt.show()



In [ ]:
pred_class = np.argmax(pred[0], axis=-1)  # shape: H x W
unique, counts = np.unique(pred_class, return_counts=True)
print(f"Prediction class distribution (slice {i}):")
for u, c in zip(unique, counts):
    print(f"Class {u}: {c} pixels")

# ampliare facendo codice che fa vedere proporzione label azzeccati su totale per ogni classe

In [ ]:
import os

save_dir = '/content/drive/MyDrive/Task01_BrainTumour/models'
os.makedirs(save_dir, exist_ok=True)

model_path = os.path.join(save_dir, '2D_Unet_Dice_crossE_25samples_8batch_4step.keras')

model.save(model_path)
print(f"Model saved to: {model_path}")
